# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 12 2022 9:50AM,246124,15,ALI023959752,"Alliance Pharma, Inc.",Released
1,Sep 12 2022 9:50AM,246124,15,30010578,"Alliance Pharma, Inc.",Released
2,Sep 12 2022 9:50AM,246124,15,ALI405532,"Alliance Pharma, Inc.",Released
3,Sep 12 2022 9:48AM,246096,21,626549,"NBTY Global, Inc.",Released
4,Sep 12 2022 9:46AM,246123,10,MSP213847,"Methapharm, Inc.",Released
5,Sep 12 2022 9:43AM,246122,12,WT5324 Out,Livs Products,Released
6,Sep 12 2022 9:41AM,246121,12,8516601,LF of America Corp.,Released
7,Sep 12 2022 9:40AM,246120,10,0085973900,ISDIN Corporation,Released
8,Sep 12 2022 9:40AM,246120,10,0085973911,ISDIN Corporation,Released
9,Sep 12 2022 9:40AM,246120,10,0085973912,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,246120,Released,60
15,246121,Released,1
16,246122,Released,1
17,246123,Released,1
18,246124,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
246120,NaN,60.0
246121,NaN,1.0
246122,NaN,1.0
246123,NaN,1.0
246124,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246096,0.0,1.0
246103,5.0,2.0
246106,0.0,1.0
246108,4.0,9.0
246109,1.0,16.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246096,0,1
246103,5,2
246106,0,1
246108,4,9
246109,1,16


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246096,0,1
1,246103,5,2
2,246106,0,1
3,246108,4,9
4,246109,1,16


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246096,,1
1,246103,5,2
2,246106,,1
3,246108,4,9
4,246109,1,16


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 12 2022 9:50AM,246124,15,"Alliance Pharma, Inc."
3,Sep 12 2022 9:48AM,246096,21,"NBTY Global, Inc."
4,Sep 12 2022 9:46AM,246123,10,"Methapharm, Inc."
5,Sep 12 2022 9:43AM,246122,12,Livs Products
6,Sep 12 2022 9:41AM,246121,12,LF of America Corp.
7,Sep 12 2022 9:40AM,246120,10,ISDIN Corporation
67,Sep 12 2022 9:11AM,246119,10,"Citieffe, Inc."
68,Sep 12 2022 9:09AM,246117,10,ISDIN Corporation
128,Sep 12 2022 9:05AM,246116,10,ISDIN Corporation
176,Sep 12 2022 9:00AM,246114,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 12 2022 9:50AM,246124,15,"Alliance Pharma, Inc.",,3
1,Sep 12 2022 9:48AM,246096,21,"NBTY Global, Inc.",,1
2,Sep 12 2022 9:46AM,246123,10,"Methapharm, Inc.",,1
3,Sep 12 2022 9:43AM,246122,12,Livs Products,,1
4,Sep 12 2022 9:41AM,246121,12,LF of America Corp.,,1
5,Sep 12 2022 9:40AM,246120,10,ISDIN Corporation,,60
6,Sep 12 2022 9:11AM,246119,10,"Citieffe, Inc.",,1
7,Sep 12 2022 9:09AM,246117,10,ISDIN Corporation,,60
8,Sep 12 2022 9:05AM,246116,10,ISDIN Corporation,,48
9,Sep 12 2022 9:00AM,246114,10,ISDIN Corporation,,60


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 9:50AM,246124,15,"Alliance Pharma, Inc.",3,
1,Sep 12 2022 9:48AM,246096,21,"NBTY Global, Inc.",1,
2,Sep 12 2022 9:46AM,246123,10,"Methapharm, Inc.",1,
3,Sep 12 2022 9:43AM,246122,12,Livs Products,1,
4,Sep 12 2022 9:41AM,246121,12,LF of America Corp.,1,
5,Sep 12 2022 9:40AM,246120,10,ISDIN Corporation,60,
6,Sep 12 2022 9:11AM,246119,10,"Citieffe, Inc.",1,
7,Sep 12 2022 9:09AM,246117,10,ISDIN Corporation,60,
8,Sep 12 2022 9:05AM,246116,10,ISDIN Corporation,48,
9,Sep 12 2022 9:00AM,246114,10,ISDIN Corporation,60,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 9:50AM,246124,15,"Alliance Pharma, Inc.",3,
1,Sep 12 2022 9:48AM,246096,21,"NBTY Global, Inc.",1,
2,Sep 12 2022 9:46AM,246123,10,"Methapharm, Inc.",1,
3,Sep 12 2022 9:43AM,246122,12,Livs Products,1,
4,Sep 12 2022 9:41AM,246121,12,LF of America Corp.,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 9:50AM,246124,15,"Alliance Pharma, Inc.",3,NaN
1,Sep 12 2022 9:48AM,246096,21,"NBTY Global, Inc.",1,NaN
2,Sep 12 2022 9:46AM,246123,10,"Methapharm, Inc.",1,NaN
3,Sep 12 2022 9:43AM,246122,12,Livs Products,1,NaN
4,Sep 12 2022 9:41AM,246121,12,LF of America Corp.,1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 9:50AM,246124,15,"Alliance Pharma, Inc.",3,0.0
1,Sep 12 2022 9:48AM,246096,21,"NBTY Global, Inc.",1,0.0
2,Sep 12 2022 9:46AM,246123,10,"Methapharm, Inc.",1,0.0
3,Sep 12 2022 9:43AM,246122,12,Livs Products,1,0.0
4,Sep 12 2022 9:41AM,246121,12,LF of America Corp.,1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1968926,255,5.0
12,984468,39,0.0
15,246124,3,0.0
19,246103,2,5.0
20,246106,1,0.0
21,246096,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1968926,255,5.0
1,12,984468,39,0.0
2,15,246124,3,0.0
3,19,246103,2,5.0
4,20,246106,1,0.0
5,21,246096,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,255,5.0
1,12,39,0.0
2,15,3,0.0
3,19,2,5.0
4,20,1,0.0
5,21,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,255.0
1,12,Released,39.0
2,15,Released,3.0
3,19,Released,2.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Executing,5.0,0.0,0.0,5.0,0.0,0.0
Released,255.0,39.0,3.0,2.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Executing,5.0,0.0,0.0,5.0,0.0,0.0
1,Released,255.0,39.0,3.0,2.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Executing,5.0,0.0,0.0,5.0,0.0,0.0
1,Released,255.0,39.0,3.0,2.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()